In [1]:
#importing packages and data
import pandas as pd
import numpy as np
prices = pd.read_csv('prices.csv')
xls = pd.ExcelFile('room_types.xlsx')
room_types = xls.parse('airbnb_room_type')
reviews = pd.read_csv('reviews.tsv', sep='\t', parse_dates=['last_review'])

In [2]:
prices.info()
room_types.info()
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   price        25209 non-null  object
 2   nbhood_full  25209 non-null  object
dtypes: int64(1), object(2)
memory usage: 591.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   description  25199 non-null  object
 2   room_type    25209 non-null  object
dtypes: int64(1), object(2)
memory usage: 591.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   listing_id   25209 non-null  int64         
 1   host_name 

In [3]:
print(prices.head())
print(room_types.head())
print(reviews.head())

   listing_id        price                nbhood_full
0        2595  225 dollars         Manhattan, Midtown
1        3831   89 dollars     Brooklyn, Clinton Hill
2        5099  200 dollars     Manhattan, Murray Hill
3        5178   79 dollars  Manhattan, Hell's Kitchen
4        5238  150 dollars       Manhattan, Chinatown
   listing_id                                description        room_type
0        2595                      Skylit Midtown Castle  Entire home/apt
1        3831            Cozy Entire Floor of Brownstone  Entire home/apt
2        5099  Large Cozy 1 BR Apartment In Midtown East  Entire home/apt
3        5178            Large Furnished Room Near B'way     private room
4        5238         Cute & Cozy Lower East Side 1 bdrm  Entire home/apt
   listing_id    host_name last_review
0        2595     Jennifer  2019-05-21
1        3831  LisaRoxanne  2019-07-05
2        5099        Chris  2019-06-22
3        5178     Shunichi  2019-06-24
4        5238          Ben  2019-06-0

In [4]:
#removing trailing dollars string from price and converting to float
prices['price'] = prices['price'].str.strip(' dollars').astype(int)
#removing outliers in price
prices = prices.loc[~prices['price'].isin([0, 7500]), :]



In [5]:
#mean rental price
prices['price'].mean()

141.52533629617872

In [6]:
#converting price to monthly
prices['monthly_price'] = prices['price']*365/12

In [7]:
#creating a separate borough column
prices['borough'] = prices['nbhood_full'].str.split(pat=', ', expand=True).get(0)


In [8]:
#cleaning room type column
room_types['room_type'].unique()
room_types['room_type'] = room_types['room_type'].str.capitalize()
room_types['room_type'] = room_types['room_type'].astype('category')

In [9]:
#merging
df = prices.merge(room_types, on='listing_id').merge(reviews, on='listing_id')

In [10]:
#dropping nulls
df = df.dropna()

In [11]:
#checking for duplicates
duplicates = df.duplicated()
df[duplicates]

,listing_id,price,nbhood_full,monthly_price,borough,description,room_type,host_name,last_review


In [16]:
#creating a price range category column
df['price_range'] = pd.cut(df['price'], bins=[0, 70,  176,  351,  np.inf], labels=['Budget','Average', 'Expensive', 'Extravagant'] )

In [13]:
print(df.head())
print(df.info())

   listing_id  price                nbhood_full  monthly_price    borough  \
0        2595    225         Manhattan, Midtown    6843.750000  Manhattan   
1        3831     89     Brooklyn, Clinton Hill    2707.083333   Brooklyn   
2        5099    200     Manhattan, Murray Hill    6083.333333  Manhattan   
3        5178     79  Manhattan, Hell's Kitchen    2402.916667  Manhattan   
4        5238    150       Manhattan, Chinatown    4562.500000  Manhattan   

                                 description        room_type    host_name  \
0                      Skylit Midtown Castle  Entire home/apt     Jennifer   
1            Cozy Entire Floor of Brownstone  Entire home/apt  LisaRoxanne   
2  Large Cozy 1 BR Apartment In Midtown East  Entire home/apt        Chris   
3            Large Furnished Room Near B'way     Private room     Shunichi   
4         Cute & Cozy Lower East Side 1 bdrm  Entire home/apt          Ben   

  last_review price_range  
0  2019-05-21   Expensive  
1  2019-07-0

In [17]:
df[df['borough']=='Bronx']['price_range'].value_counts()

Budget         393
Average        273
Expensive       25
Extravagant      5
Name: price_range, dtype: int64

In [15]:
df.to_csv('cleaned.csv', index='False')